In [40]:
import pandas as pd
import datetime
import time
import math

In [41]:
folder = '07_12_2018_09_51'
res_adapt = pd.read_csv(folder + '/adapt.csv')
res_adapt_eval = pd.read_csv(folder + '/adapt_eval.csv')
res_eval = pd.read_csv(folder + '/eval.csv')

In [42]:
res_adapt.drop(res_adapt[res_adapt.instance.str.contains('exact')].index, inplace=True)
res_eval.drop(res_eval[res_eval.instance.str.contains('exact')].index, inplace=True)
res_adapt_eval.drop(res_adapt_eval[res_adapt_eval.instance.str.contains('exact')].index, inplace=True)

In [43]:
res_eval['diff_p'] = (res_eval['pred_p'] - res_eval['real_p'])
res_eval['diff_p'] = res_eval['diff_p']*res_eval['diff_p']

In [44]:
res_eval.groupby('type')[['diff_p']].mean()

,diff_p
type,
CNN,5.606131e-01
CRNN,5.164956e-05
DNN,7.142669e-02
FeatureBased_Bayesian Ridge Regression,5.235106e-05
FeatureBased_Lasso Regression,5.235106e-05
FeatureBased_Linear Regression,5.235106e-05
FeatureBased_Passive Aggressive Regression,1.447790e-04
FeatureBased_SGD Regression,3.595260e+18
FeatureBased_SVM,5.235106e-05


In [45]:
res_eval.groupby('type')[['real_p','pred_p']].corr().iloc[0::2,-1]

type                                              
CNN                                         real_p    1.960262e-02
CRNN                                        real_p    1.184513e-01
DNN                                         real_p    5.546669e-02
FeatureBased_Bayesian Ridge Regression      real_p   -8.867088e-15
FeatureBased_Lasso Regression               real_p   -8.867088e-15
FeatureBased_Linear Regression              real_p   -8.867088e-15
FeatureBased_Passive Aggressive Regression  real_p             NaN
FeatureBased_SGD Regression                 real_p   -6.107902e-01
FeatureBased_SVM                            real_p   -8.794758e-15
FeatureBased_Theil-Sen Regression           real_p   -6.109572e-01
GRU                                         real_p   -1.661888e-02
Name: pred_p, dtype: float64

In [46]:
res_adapt['diff_p'] = res_adapt['new_p'] - res_adapt['old_p']
res_adapt['diff_r'] = res_adapt['new_r'] - res_adapt['old_r']
res_adapt['diff_f'] = res_adapt['new_f'] - res_adapt['old_f']

In [47]:
res_adapt.groupby('type').mean()

,Unnamed: 0,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
type,,,,,,,,,,,
BPR,4972,0,0.005235,0.5,0.010323,0.005235,0.500000,0.010323,0.000000,0.000000,0.000000
CNN,4974,0,0.005235,0.5,0.010323,0.657810,0.614591,0.625985,0.652574,0.114591,0.615662
CRNN,4976,0,0.005235,0.5,0.010323,0.658077,0.614535,0.625856,0.652842,0.114535,0.615533
DNN,4975,0,0.005235,0.5,0.010323,0.658265,0.605640,0.619736,0.653030,0.105640,0.609413
GRU,4973,0,0.005235,0.5,0.010323,0.527742,0.524818,0.523565,0.522507,0.024818,0.513242
IR,4970,0,0.005235,0.5,0.010323,0.592200,0.532474,0.544576,0.586964,0.032474,0.534252
SVDpp,4971,0,0.005235,0.5,0.010323,0.502153,0.502680,0.501798,0.496918,0.002680,0.491475


In [24]:
res_adapt

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,diff_p,diff_r,diff_f
0,0,1 AMC Token Path Ontobuilder MWBG,IR,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,-0.082249,-0.218658,-0.115527
1,1,1 AMC Token Path Ontobuilder MWBG,SVDpp,0,0.578630,0.718658,0.613711,0.996708,0.545455,0.581682,0.418078,-0.173203,-0.032029
2,2,1 AMC Token Path Ontobuilder MWBG,BPR,0,0.578630,0.718658,0.613711,0.125000,0.000083,0.000166,-0.453630,-0.718575,-0.613545
3,3,1 AMC Token Path Ontobuilder MWBG,GRU,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,-0.082249,-0.218658,-0.115527
4,4,1 AMC Token Path Ontobuilder MWBG,CNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,-0.082249,-0.218658,-0.115527
5,5,1 AMC Token Path Ontobuilder MWBG,DNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,-0.082249,-0.218658,-0.115527
6,6,1 AMC Token Path Ontobuilder MWBG,CRNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,-0.082249,-0.218658,-0.115527
7,7,1 Ontobuilder Term Match Ontobuilder Threshold,IR,0,0.697030,0.589915,0.623016,0.496382,0.500000,0.498184,-0.200648,-0.089915,-0.124832
8,8,1 Ontobuilder Term Match Ontobuilder Threshold,SVDpp,0,0.697030,0.589915,0.623016,0.496370,0.498343,0.497354,-0.200660,-0.091572,-0.125661
9,9,1 Ontobuilder Term Match Ontobuilder Threshold,BPR,0,0.697030,0.589915,0.623016,0.126302,0.022893,0.002794,-0.570728,-0.567022,-0.620222


In [37]:
res_adapt_eval['pred_e_max'] = res_adapt_eval.groupby(['instance','type'])['real_e'].transform(max)
res_adapt_eval['k_max'] = res_adapt_eval.groupby(['instance','type'])['k'].transform(max)
clean_a_e = res_adapt_eval[(res_adapt_eval['pred_e_max'] == res_adapt_eval['real_e']) &
               (res_adapt_eval['k_max'] == res_adapt_eval['k'])]

In [38]:
res_adapt_eval

,Unnamed: 0,instance,type,k,old_p,old_r,old_f,new_p,new_r,new_f,real_e,pred_e,pred_e_max,k_max
0,0,1 AMC Token Path Ontobuilder MWBG,GRU_GRU,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.643926,0.613711,1
1,1,1 AMC Token Path Ontobuilder MWBG,GRU_GRU,1,0.496382,0.500000,0.498184,0.496382,0.500000,0.498184,0.613711,0.643931,0.613711,1
2,2,1 AMC Token Path Ontobuilder MWBG,CNN_CNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.224611,0.613711,0
3,3,1 AMC Token Path Ontobuilder MWBG,DNN_DNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.641835,0.613711,0
4,4,1 AMC Token Path Ontobuilder MWBG,CRNN_CRNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.685498,0.613711,0
5,5,1 AMC Token Path Ontobuilder MWBG,GRU_CNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.224611,0.613711,0
6,6,1 AMC Token Path Ontobuilder MWBG,GRU_DNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.641835,0.613711,0
7,7,1 AMC Token Path Ontobuilder MWBG,GRU_CRNN,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.685498,0.613711,0
8,8,1 AMC Token Path Ontobuilder MWBG,CNN_GRU,0,0.578630,0.718658,0.613711,0.496382,0.500000,0.498184,0.613711,0.643926,0.613711,1
9,9,1 AMC Token Path Ontobuilder MWBG,CNN_GRU,1,0.496382,0.500000,0.498184,0.496382,0.500000,0.498184,0.613711,0.643931,0.613711,1


In [39]:
clean_a_e.groupby('type').mean()[['new_p', 'new_r', 'new_f']]

,new_p,new_r,new_f
type,,,
BPR_FEATBayesian Ridge Regression,0.510157,0.572099,0.491220
BPR_FEATLasso Regression,0.510157,0.572099,0.491220
BPR_FEATLinear Regression,0.510157,0.572099,0.491220
BPR_FEATPassive Aggressive Regression,0.510157,0.572099,0.491220
BPR_FEATSGD Regression,0.510157,0.572099,0.491220
BPR_FEATSVM,0.510157,0.572099,0.491220
BPR_FEATTheil-Sen Regression,0.510157,0.572099,0.491220
BPR_GRU,0.510157,0.572099,0.491220
CNN_CNN,0.548107,0.543853,0.528630


In [34]:
def summerize(data, file):
    idMax = data[data['type'] == data['type']]
    if 'pred_e' in data:
        idMax = data.groupby(['type'])['pred_e'].transform(max) == data['pred_e']
        pd.DataFrame(data[idMax].groupby(by='type').mean()).to_csv(file)
    else:
        pd.DataFrame(data.groupby(by='type').mean()).to_csv(file)

In [35]:
data = pd.read_csv('adapt.csv', low_memory=False)
summerize(data, 'a.csv')

FileNotFoundError: File b'adapt.csv' does not exist